In [7]:
import re
import pdfminer
from subprocess import check_output
from bs4 import BeautifulSoup
from collections import defaultdict

In [2]:
# install pdfminer with this:
# pip3 install pdfminer.six

In [3]:
# run this in shell to grab file:
# curl "https://madison.legistar.com/View.ashx?M=A&ID=782498&GUID=95864EAA-10C1-4866-AA34-C134B80BA5F5" -o agenda.pdf

In [8]:
s = check_output(["pdf2txt.py", "-t", "html", "-Y", "loose", "agenda.pdf"])
# loose vs exact

In [19]:
page = BeautifulSoup(s)
page

<html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:612px; height:792px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:244px; top:100px; width:119px; height:16px;"><span style="font-family: Arial,Bold; font-size:16px">City of Madison
</span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:238px; top:141px; width:131px; height:14px;"><span style="font-family: Arial,Bold; font-size:14px">Agenda - Approved
</span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:455px; top:99px; width:86px; height:30px;"><span style="font-family: Arial; font-size:8px">City of Madison
Madison, WI  53703
www.cityofmadison.com
</span></div><div style="position:absolute; border: textbox 1px so

In [24]:
page.find_all("div")[50]

<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:168px; top:1509px; width:122px; height:22px;"><span style="font-family: Arial; font-size:9px">Wilson Street Corridor Plan 
 
</span></div>

In [13]:
text = {} # key=(x,y), val=text

for div in page.find_all("div"): #gets a list of different div tags 
    style = div.attrs.get("style", "") #attrs will tell you the positioning
    m1 = re.search("left:(\d+)px", style)
    m2 = re.search("top:(\d+)px", style)
    if m1 and m2:
        x, y = int(m1.group(1)), int(m2.group(1))#each parenthesis is a different group
        text[(x,y)] = div.get_text()

In [14]:
text

{(244, 100): 'City of Madison\n',
 (238, 141): 'Agenda - Approved\n',
 (455, 99): 'City of Madison\nMadison, WI  53703\nwww.cityofmadison.com\n',
 (192, 165): 'TRANSPORTATION POLICY AND \nPLANNING BOARD\n',
 (54, 213): 'Monday, May 4, 2020\n',
 (293, 213): '6:00 PM\n',
 (491, 213): 'Virtual Meeting\n',
 (91,
  240): 'The City of Madison is holding the Transportation Policy and Planning Board meeting \nvirtually to help protect our community from the Coronavirus (COVID-19) pandemic.\n',
 (168,
  285): 'The City of Madison is holding the Transportation Policy and Planning Board meeting \nvirtually to help protect our community from the Coronavirus (COVID-19) pandemic.\n',
 (168,
  321): '1. WRITTEN COMMENTS: You can send comments on agenda items to \naschroeder@cityofmadison.com \n',
 (168,
  357): '2. REGISTER BUT DO NOT SPEAK: You can register your support or opposition to an \nagenda item at https://www.cityofmadison.com/MeetingRegistration.\n',
 (168,
  393): '3. REGISTER TO SPEAK OR

In [31]:
def find_near(x, y):
    for x_ in range(x-5, x+5):
        for y_ in range(y-5, y+5):
            if (x_, y_) in text:
                return text[(x_, y_)]
    return None

agenda_list = {}

for (x,y), t in text.items():
    # agenda items are roughly at x=170
    if 165 < x < 175:
        # agenda numbers are rougly at x=69, same y as text
        num = find_near(69, y)
        if num:
            agenda_list[int(num.split(".")[0])] = t

agenda_list

{3: 'Opening Comments - Mayor Satya Rhodes-Conway (10 minutes)\nMetro Route Restructure, Complete Streets, Parking Ordinance and TOD \nOverlay, BRT\n \n',
 4: 'Introduction of New Transit General Manager - Tom Lynch, Justin Stuehrenberg \n(10 minutes)\n \n',
 5: 'Adopting the Oscar Mayer Special Area Plan as a Supplement to the City of \nMadison Comprehensive Plan.\n',
 6: 'Amending the timeline for final Staff Team recommendations on parking \npolicies as adopted by RES-19-00321 (Legistar File ID 54961) on 4/16/2019.\n',
 7: 'Authorizing the Mayor and the City Clerk to enter into a project agreement with \nthe Wisconsin Department of Transportation (WisDOT) and other project \npartners to construct an interchange at USH 12/18 and County AB. (16 AD)\n',
 8: 'Metro Status and Update on Metro Restructure RFP - Mike Cechvala\n \n',
 9: 'COVID-19 Transportation Items - Tom Lynch, Sabrina Tolley\n \n',
 10: 'Wilson Street Corridor Plan \n \n',
 11: 'Vehicle Miles Traveled and Mobility Initi

In [33]:
print(agenda_list[3])

Opening Comments - Mayor Satya Rhodes-Conway (10 minutes)
Metro Route Restructure, Complete Streets, Parking Ordinance and TOD 
Overlay, BRT
 

